In [1]:
import pandas as pd
import polars as pl
import numpy as np
import os, gc
from tqdm.auto import tqdm
from matplotlib import pyplot as plt
import pickle

import torch
import torch.nn as nn
import torch.nn.functional as F
from pytorch_lightning import (LightningDataModule, LightningModule, Trainer)
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint, Timer

import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader


from sklearn.metrics import r2_score
from lightgbm import LGBMRegressor
import lightgbm as lgb
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import VotingRegressor

import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None



/Users/apple/miniconda3/envs/quant2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
class CONFIG:
    seed = 42
    target_col = "responder_6"
    # feature_cols = ["symbol_id", "time_id"] + [f"feature_{idx:02d}" for idx in range(79)]+ [f"responder_{idx}_lag_1" for idx in range(9)]
    feature_cols = [f"feature_{idx:02d}" for idx in range(79)]+ [f"responder_{idx}_lag_1" for idx in range(9)]
    
    model_paths = [
        "/Users/apple/Masters/Job/kaggle/jane-street-real-time-market-data-forecasting/NN+GBDT/GBDT_result.pkl"
        "/Users/apple/Masters/Job/kaggle/jane-street-real-time-market-data-forecasting/nn_0.model",
    ]

In [3]:
valid = pl.scan_parquet(
    f"/Users/apple/Masters/Job/kaggle/jane-street-real-time-market-data-forecasting/validation.parquet"
).collect().to_pandas()

In [ ]:
N_folds = 1
# 加载最佳模型
models = []
for fold in range(N_folds):
    checkpoint_path = f"{CONFIG.model_paths[0]}"

    models.append(model.to("cpu"))

In [ ]:
models[0]

# CV Score

In [4]:
X_valid = valid[ CONFIG.feature_cols ]
y_valid = valid[ CONFIG.target_col ]
w_valid = valid[ "weight" ]
X_valid = X_valid.fillna(method = 'ffill').fillna(0)
X_valid.shape, y_valid.shape, w_valid.shape

((1082224, 88), (1082224,), (1082224,))

In [ ]:
y_pred_valid = np.zeros(y_valid.shape)
with torch.no_grad():
    for model in models:
        model.eval()
        y_pred_valid += model(torch.FloatTensor(X_valid.values).to("cpu")).cpu().numpy() / len(models)
valid_score = r2_score( y_valid, y_pred_valid, sample_weight=w_valid )
valid_score

In [ ]:
del valid, X_valid, y_valid, w_valid
gc.collect()